In [ ]:
pip install unidecode

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.9/235.9 KB 5.0 MB/s eta 0:00:00


In [ ]:
pip install sumy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.3/97.3 KB 2.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 78.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
  Created wheel for breadability: filename=breadability-0.1.20-py2.py3-none-any.whl size=21714 sha256=57189594e22637d6974d5aae9fd0b0e8fb2a2a7669404eed8ea717b2dcd59d3f
  Stored in directory: /root/.cache/pip/wheels/5f/0d/0c/2062d8c1758b4b1a2e42b4a63e6660d9ec2ba9463cfee9eeab
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13723 sha256=9e6d5d053f8040ac9f6d1eed2077dbcd693e429350b77ae47db603cb58295d49
  Stored in directory: /root/.cache/pip/wheels/56/ea/58/ead137b087d9e326852a851351d1debf4ada529

In [ ]:
from bs4 import BeautifulSoup
import requests
import tweepy
import json
import pandas as pd
import os
from time import sleep
import re 
from bs4.element import Comment
from unidecode import unidecode
import nltk
import heapq
from lxml import html
from lxml import etree
from datetime import datetime

from sumy.summarizers.lsa import LsaSummarizer
from sumy.nlp.tokenizers import Tokenizer
from sumy.parsers.plaintext import PlaintextParser

In [ ]:
class Events():
    def __init__(self, name, link, date, hora, location):
        self.name = name
        self.link = link
        self.date = date
        self.hora = hora
        self.location = location

    def GetKeywords(self):
        l = []
        if self.link != '':
            l += [self.link, self.link.replace(" ","")]  
        #set to avoid posible duplicates 
        return set(l)
    
    #usefull for the creation of a DataFrame later on
    def __dict__(self):
        return {
            "Nombre del Evento" : self.name,
            "Link Inscripcion" : self.link,
            "Fecha" : self.date,
            "Hora" : self.hora,
            "Ubicacion" : self.location,
        }
    
    def __str__(self):
        return "\nNombre del Evento: "+ self.name \
                + "\nLink Inscripcion: " + self.link \
                +"\nFecha: " + self.date \
                +"\nHora: " + self.hora\
                +"\nUbicacion: " + self.location\
                + "\n"

In [ ]:

r = requests.get("https://www.eventbrite.es/d/spain--madrid/technology/")
soup = BeautifulSoup(r.content,"html.parser")

n=soup.find_all("div",{"class":"eds-event-card-content__content__principal"})

events=[]
for i in n:
  names= i.find("div", {"class": "eds-event-card__formatted-name--is-clamped eds-event-card__formatted-name--is-clamped-three eds-text-weight--heavy"}).text
  links= i.find("a")["href"]
  d= i.find("div", {"class": "eds-event-card-content__sub-title eds-text-color--primary-brand eds-l-pad-bot-1 eds-l-pad-top-2 eds-text-weight--heavy eds-text-bm"}).text.split("+")
  d1 = d[0].replace("hoy a las", ",").replace(".,", ".")
  d2= d1.split(",")
  date= d2[0]
  hora = d2[1]
  location= i.find("div", {"class":"card-text--truncated__one"}).text
  events.append(Events(names,links,date,hora, location))



In [ ]:
currentMonth = datetime.now().month
nextMonth = datetime.now().month+1


In [ ]:
dfEvents = pd.DataFrame([o.__dict__() for o in events])
dfEvents = dfEvents[(dfEvents["Fecha"] != "" )]
dfEvents

,Nombre del Evento,Link Inscripcion,Fecha,Hora,Ubicacion
0,100Startups Health [Professional],https://www.eventbrite.es/e/entradas-100startu...,jue. mar. 23,8:30,La Nave • Madrid
1,100Startups Health [Professional],https://www.eventbrite.es/e/entradas-100startu...,jue. mar. 23,8:30,La Nave • Madrid
2,Pulse — Renewable energy event,https://www.eventbrite.es/e/pulse-renewable-en...,mié. abr. 19,9:00,Palacio Neptuno • Madrid
3,Pulse — Renewable energy event,https://www.eventbrite.es/e/pulse-renewable-en...,mié. abr. 19,9:00,Palacio Neptuno • Madrid
4,WORLD ENGINEERING DAY 2023,https://www.eventbrite.es/e/world-engineering-...,jue. mar. 2,8:30,Instituto de la Ingeniería de España • Madrid
5,WORLD ENGINEERING DAY 2023,https://www.eventbrite.es/e/world-engineering-...,jue. mar. 2,8:30,Instituto de la Ingeniería de España • Madrid
8,Telemedicine Entrepreneurship Training & Works...,https://www.eventbrite.com/e/telemedicine-entr...,mar. feb. 28,13:00,Madrid • Madrid
9,Telemedicine Entrepreneurship Training & Works...,https://www.eventbrite.com/e/telemedicine-entr...,mar. feb. 28,13:00,Madrid • Madrid
10,IE University Marketing Challenge: The beauty ...,https://www.eventbrite.es/e/ie-university-mark...,mar. mar. 28,15:30,Hybrid: Online or IE TOWER - 9th Floor • Madrid
11,IE University Marketing Challenge: The beauty ...,https://www.eventbrite.es/e/ie-university-mark...,mar. mar. 28,15:30,Hybrid: Online or IE TOWER - 9th Floor • Madrid


Guardar como csv en drive

In [ ]:
from google.colab import drive

drive.mount('/content/drive')
path = '/content/drive/My Drive/adchase_events_1.csv'  #CAMBIAR NOMBRE ARCHIVO
with open(path, 'w', encoding = 'utf-8-sig') as f:
  dfEvents.to_csv(f)

Mounted at /content/drive
